check

netstat -an | find "8081"

run cmd1: 

flower-superlink --insecure --fleet-api-address="localhost:8081"

run cmd2:

flower-supernode --insecure --superlink="localhost:8081"

then run the code

In [5]:
!pip install ollama
!pip install gradio
!pip install langchain
!pip install torch
!pip install flwr
!pip install numpy
!pip install fastapi

  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
Using cached typer-0.15.1-py3-none-any.whl (44 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flwr 1.11.1 requires typer[all]<0.10.0,>=0.9.0, but you have typer 0.15.1 which is incompatible.


  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)
Using cached typer-0.9.4-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.15.1
    Uninstalling typer-0.15.1:
      Successfully uninstalled typer-0.15.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.44.1 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.


Found existing installation: typer 0.9.4
Uninstalling typer-0.9.4:
  Successfully uninstalled typer-0.9.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.44.1 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.0 which is incompatible.


In [6]:
import ollama
import gradio as gr
import re
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Tuple

# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.logsoftmax(x)

def load_data() -> Tuple[torch.utils.data.DataLoader, torch.utils.data.DataLoader]:
    # Implement a data loading function for your dataset
    pass

# Flower client definition
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model):
        self.model = model

    def get_parameters(self) -> List[np.ndarray]:
        return [val.cpu().numpy() for val in self.model.parameters()]

    def set_parameters(self, parameters: List[np.ndarray]) -> None:
        for val, param in zip(parameters, self.model.parameters()):
            param.data = torch.tensor(val)

    def fit(self, parameters: List[np.ndarray], config: dict) -> Tuple[List[np.ndarray], int, dict]:
        self.set_parameters(parameters)
        # Implement your training function
        pass

    def evaluate(self, parameters: List[np.ndarray], config: dict) -> Tuple[float, int, dict]:
        self.set_parameters(parameters)
        # Implement your evaluation function
        pass

# Function to process text
def process_text(input_data):
    if input_data is None:
        return None, None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_text(input_data)
    return text_splitter, chunks

# Function to interact with the LLM
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model="aidoctor", messages=[{'role': 'user', 'content': formatted_prompt}])
    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>', '', response_content, flags=re.DOTALL).strip()
    return final_answer

# Function to run RAG chain
def rag_chain(question, chunks):
    formatted_content = "\n\n".join(chunks)
    return ollama_llm(question, formatted_content)

# Main function for detailed diagnosis
def detailed_diagnosis(input_text):
    text_splitter, chunks = process_text(input_text)
    if text_splitter is None:
        return "No text input provided."

    diagnosis = rag_chain("Please provide a detailed diagnosis for the symptoms described.", chunks)
    cures = rag_chain("What are the possible cures for the diagnosed condition?", chunks)
    treatments = rag_chain("What are the possible treatments for the diagnosed condition?", chunks)
    diagnostic_tests = rag_chain("What diagnostic tests should be conducted based on the symptoms?", chunks)
    specialists = rag_chain("Which specialists should be consulted for the symptoms?", chunks).split('.')[0]
    monitoring = rag_chain("How should the symptoms be monitored?", chunks).split('.')[0]
    disease_info = rag_chain("Please provide detailed information about the diagnosed disease.", chunks).split('.')[0]

    structured_response = (
        f" Diagnosis:\n\n{diagnosis}\n\n"
        f" Cures:\n\n{cures}\n\n"
        f" Treatments:\n\n{treatments}\n\n"
        f" Diagnostic Tests:\n\n{diagnostic_tests}\n\n"
        f" Other Details:\n\n"
        f"- Specialists: {specialists}.\n"
        f"- Monitoring: {monitoring}.\n"
        f"- Disease Information: {disease_info}.\n"
    )

    return structured_response

# Set up Flower client for federated learning
model = Net()
client = FlowerClient(model)

# Define a Gradio interface
interface = gr.Interface(
    fn=detailed_diagnosis,
    inputs=gr.Textbox(lines=10, label="Describe your symptoms"),
    outputs="text",
    title="AI Doctor - Detailed Diagnosis",
    description="Enter your symptoms to receive a detailed medical diagnosis and recommendations."
)

interface.launch()


C:\Users\gurba\anaconda3\envs\myResearchEnv\lib\site-packages\gradio\routes.py:1215: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
C:\Users\gurba\anaconda3\envs\myResearchEnv\lib\site-packages\fastapi\applications.py:4495: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  return self.router.on_event(event_type)
C:\Users\gurba\anaconda3\envs\myResearchEnv\lib\site-packages\gradio\route_utils.py:871: DeprecationWarning: The loop argument is deprecated since Python 3.8, and scheduled for removal in Python 3.10.
  app.stop_event = asyncio.Event(loop=loop)


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


C:\Users\gurba\anaconda3\envs\myResearchEnv\lib\site-packages\starlette\templating.py:161: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(


I have been experiencing the following symptoms:
- Persistent headache
- Nausea and vomiting
- Sensitivity to light
- Stiff neck
- Fever

Please provide a detailed diagnosis, possible cures, treatments, recommended diagnostic tests, specialists to consult, how to monitor the symptoms, and additional information about the disease.
